보통 프로젝트를 진행할때, 소스 버전 컨트롤을 위해서 git을 많이 사용합니다. git을 로컬에서 사용하기 보다는 github 또는 gitlab을 주로 이용하게 되는데요.

github 및 gitlab 사용시 인증은 간편하게 http 인증을 많이 사용했었는데요.

프로젝트가 많아짐에 따라 commit 할때마다 비번을 넣어야 하는 불편함과 한 컴퓨터에서 gitlab, github내에 여러 계정이 혼재 할 경우 http 인증이 동작하지 않아 다른 방법을 찾다가 SSH Key를 이용한 인증 방법을 사용하면 해결이 가능하다는 것을 알게 되었습니다.

<!-- TEASER_END -->

먼저 sshkey-gen 명령을 통해서 각 git 계정 마다 사용할 키를 생성해줍니다.
A계정은 개인 소스 저장소용이고, B계정은 B회사 소스 저장소용, C계정은 C회사 소스 저자용이라고 하겠습니다. 

```bash
ssh-keygen -t rsa -b 4096 -C "A@A.com"
Generating public/private rsa key pair.
Enter file in which to save the key (~/.ssh/id_rsa): a_id_rsa

ssh-keygen -t rsa -b 4096 -C "B@B.com"
Generating public/private rsa key pair.
Enter file in which to save the key (~/.ssh/id_rsa): b_id_rsa

ssh-keygen -t rsa -b 4096 -C "C@C.com"
Generating public/private rsa key pair.
Enter file in which to save the key (~/.ssh/id_rsa): c_id_rsa

```

키를 생성하게 되면 각각 개인키와 공개키가 생성되게 됩니다.

* 개인키: a_id_rsa, b_id_rsa, c_id_rsa
* 공개키: a_id_rsa.pub, b_id_rsa.pub, c_id_rsa.pub

계정의 .ssh 디렉토리에 개인키와 공개키를 복사합니다.

이제 생성된 개인키를 SSH 에 등록해 줍니다.

```bash
ssh-add ~/.ssh/a_id_rsa
ssh-add ~/.ssh/b_id_rsa
ssh-add ~/.ssh/c_id_rsa
```

등록된 키는 ssh-add -l 명령어로 확인이 가능합니다.

이제 생성된 공개키를 각 사이트 계정의 SSH Key 등록하는 곳에 등록해 주면 됩니다.

SSH Key를 등록하는 방법은 아래 링크를 참조하세요.

* Github: https://help.github.com/articles/adding-a-new-ssh-key-to-your-github-account/
* Gitlab: https://about.gitlab.com/2014/03/04/add-ssh-key-screencast/ 

이제 SSH를 이용한 접속시 자동으로 해당 키를 사용할 수 있도록 설정을 해줘야 합니다.

~/.ssh/config 파일을 생성해서 아래와 같이 설정해 줍니다.

```bash
Host A.github.com
        HostName github.com
        User A
        PreferredAuthentications publickey
        IdentityFile ~/.ssh/a_id_rsa
Host B.gitlab.com
        HostName gitlab.com
        User B
        PreferredAuthentications publickey
        IdentityFile ~/.ssh/b_id_rsa
Host C.gitlab.com
        HostName gitlab.com
        User C
        PreferredAuthentications publickey
        IdentityFile ~/.ssh/c_id_rsa
```

이제 git 레포지토리로 이동하여 git config에 .git/config파일에 Host부분을 ~/.ssh/config 에 설정한 Host와 일치 시켜 줍니다.

```bash
[remote "origin"]
        url = git@A.github.com:comafire/test.git
        fetch = +refs/heads/*:refs/remotes/origin/*
```

이제 git 사용시 각 계정 마다 다른 키 파일을 이용하여 암호를 넣지 않고 편리하게 사용 할 수 있습니다.